In [1]:
import ollama

# Ensure the environment variable is set before running Python
import os
os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11500"

# Now, interact with the model
response = ollama.chat(
    model='mistral',

    messages=[{'role': 'user', 'content': 'do you know me?'}]
)

print(response['message']['content'])


 I don't have personal experiences or memories, so I don't "know" people in the same way humans do. However, I can interact with and provide assistance to many different users based on their questions and requests. If you would like to tell me more about yourself, we can use that information to better tailor my responses to your needs.


In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mae
from tensorflow.keras.metrics import Mean
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# Load and preprocess the dataset
df = pd.read_csv("E:\TextMining\Shahrin LLM\descriptive_patient_dataset.csv")
df[:2]

Patient Name    f1: Frequency f2: Urgency f3: Severity       f4: Capacity  \
0  Patient_0001             High    Moderate     Moderate      Full Capacity   
1  Patient_0002  Low to Moderate         Low    Very High  Slightly Impaired   

       h1: Age h2: Gender                 h3: Education Level  \
0  Older Adult     Female  Some college / Vocational training   
1  Middle-Aged     Female  Some college / Vocational training   

  h4: Health Literacy h5: Tech Proficiency h6: Language / Culture  \
0                 Low             Very Low     Very Low Alignment   
1                 Low             Moderate                    Low   

  h7: Socioeconomic Status h8: Previous Experience h9: Cognitive/Mental State  \
0                     High                Moderate         Clear but Stressed   
1                      Low                Moderate        Fluctuating / Mixed   

  f6: Redundancy f7: Environment f8: Communication Quality  \
0            Low        Moderate                  Very Low   
1       Moderate        Moderate                       Low   

  f9: Consent Granularity f10: Cumulative Load Trigger Type  
0                    High            Very High      Dynamic  
1               Very High            Very High      Dynamic

In [3]:
df.columns
#f2, f4, h1, h6, h9

Index(['Patient Name', 'f1: Frequency', 'f2: Urgency', 'f3: Severity',
       'f4: Capacity', 'h1: Age', 'h2: Gender', 'h3: Education Level',
       'h4: Health Literacy', 'h5: Tech Proficiency', 'h6: Language / Culture',
       'h7: Socioeconomic Status', 'h8: Previous Experience',
       'h9: Cognitive/Mental State', 'f6: Redundancy', 'f7: Environment',
       'f8: Communication Quality', 'f9: Consent Granularity',
       'f10: Cumulative Load', 'Trigger Type'],
      dtype='object')

In [4]:
df=df[['f2: Urgency','f4: Capacity','h1: Age','h6: Language / Culture','h9: Cognitive/Mental State','Trigger Type']]

In [5]:
df['Trigger Type'].value_counts()

Trigger Type
Dynamic        500
Non-Dynamic    500
Name: count, dtype: int64

In [6]:
import pandas as pd

# Example replacements
df['Trigger Type'] = df['Trigger Type'].replace({
    'Non-Dynamic': 'Static'
})

In [7]:
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
df_shuffled[:2]

f2: Urgency       f4: Capacity      h1: Age h6: Language / Culture  \
0         Low  Slightly Impaired  Older Adult               Moderate   
1   Very High  Slightly Impaired  Middle-Aged              Very High   

  h9: Cognitive/Mental State Trigger Type  
0         Clear but Stressed       Static  
1        Fluctuating / Mixed      Dynamic

In [9]:
import pandas as pd


# Function to convert each row into a formatted prompt
def generate_prompt(row):
    prompt = f"""### Instruction:
Based on the following patient data, classify the appropriate Trigger Type (e.g., Dynamic, Static) that best describes the situation.

### Input:
f2: Urgency - {row['f2: Urgency']}
f4: Capacity - {row['f4: Capacity']}
h1: Age - {row['h1: Age']}
h6: Language / Culture - {row['h6: Language / Culture']}
h9: Cognitive/Mental State - {row['h9: Cognitive/Mental State']}

### Response:
Trigger Type: {row['Trigger Type']}
"""
    return prompt

df1=df_shuffled[:500]
# Generate prompts for each row
df1['prompt'] = df1.apply(generate_prompt, axis=1)

# Save as .jsonl (useful for training with Hugging Face or LLMs)
#df1[['prompt']].to_json("trigger_prompts.jsonl", orient='records', lines=True)

# Optional: Print sample prompt
#print(df1['prompt'].iloc[0])


In [10]:
len(df1)

500

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

# Create vector embeddings from the formatted samples
embeddings = model.encode(df1['prompt'], show_progress_bar=True)


import faiss
import numpy as np

embedding_matrix = np.array(embeddings).astype('float32')

# Create index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)



Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [12]:
embeddings.shape

(500, 768)

In [14]:
import pandas as pd


# Function to convert each row into a formatted prompt
def generate_prompt1(row):
    prompt = f"""### Instruction:
Based on the following patient data, classify the appropriate Trigger Type (e.g., Dynamic, Static) that best describes the situation.

### Input:
f2: Urgency - {row['f2: Urgency']}
f4: Capacity - {row['f4: Capacity']}
h1: Age - {row['h1: Age']}
h6: Language / Culture - {row['h6: Language / Culture']}
h9: Cognitive/Mental State - {row['h9: Cognitive/Mental State']}

"""
    return prompt

df2=df_shuffled[600:650]
# Generate prompts for each row
df2['prompt'] = df2.apply(generate_prompt1, axis=1)


# Optional: Print sample prompt
#print(df2['prompt'].iloc[0])


In [16]:
df2[:2]

f2: Urgency         f4: Capacity      h1: Age h6: Language / Culture  \
600    Moderate  Moderately Impaired  Middle-Aged               Moderate   
601    Very Low        Full Capacity  Older Adult                   High   

    h9: Cognitive/Mental State Trigger Type  \
600         Clear but Stressed      Dynamic   
601           Stable and Clear       Static   

                                                prompt  
600  ### Instruction:\nBased on the following patie...  
601  ### Instruction:\nBased on the following patie...

In [17]:
def create_rag_prompt(sample, k=5):
    query_text = generate_prompt1(sample)
    query_vec = model.encode([query_text]).astype('float32')

    # Find top-k similar samples
    D, I = index.search(query_vec, k)
    context = "\n".join([df1['prompt'].iloc[i] for i in I[0]])

    # Final prompt
    return f"""Here are examples of patient profiles:\n\n{context}\n\nNow classify the following sample:\n{query_text}\n\n Answer as: "Dynamic" or "Static"."""


In [18]:
create_rag_prompt(df2.iloc[1])

'Here are examples of patient profiles:\n\n### Instruction:\nBased on the following patient data, classify the appropriate Trigger Type (e.g., Dynamic, Static) that best describes the situation.\n\n### Input:\nf2: Urgency - Very Low\nf4: Capacity - Full Capacity\nh1: Age - Older Adult\nh6: Language / Culture - Moderate\nh9: Cognitive/Mental State - Stable and Clear\n\n### Response:\nTrigger Type: Static\n\n### Instruction:\nBased on the following patient data, classify the appropriate Trigger Type (e.g., Dynamic, Static) that best describes the situation.\n\n### Input:\nf2: Urgency - Very Low\nf4: Capacity - Full Capacity\nh1: Age - Older Adult\nh6: Language / Culture - High\nh9: Cognitive/Mental State - Stable and Clear\n\n### Response:\nTrigger Type: Static\n\n### Instruction:\nBased on the following patient data, classify the appropriate Trigger Type (e.g., Dynamic, Static) that best describes the situation.\n\n### Input:\nf2: Urgency - Very Low\nf4: Capacity - Full Capacity\nh1: Ag

In [19]:
import time
import psutil
import GPUtil
import ollama

# Function to monitor CPU and RAM
def log_cpu_memory_usage():
    cpu = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory().percent
    print(f"CPU: {cpu}% | RAM: {memory}%")

# Function to monitor GPU (NVIDIA)
def log_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        for gpu in gpus:
            print(f"GPU: {gpu.name} | Memory Used: {gpu.memoryUsed}MB / {gpu.memoryTotal}MB | Load: {gpu.load * 100:.1f}%")
    else:
        print("No GPU detected or not supported.")

# Your original timing code
start_time = time.time()
pred = []

for i in range(len(df2)):
    response = ollama.chat(
        model="phi",
        messages=[{"role": "user", "content": create_rag_prompt(df2.iloc[i])}]
    )
    prediction = response['message']['content']
    pred.append(prediction)
    print(f"Processed row {i}")

    # Log resource usage after each prediction
    log_cpu_memory_usage()
    log_gpu_usage()

print("--- Total time: %.2f seconds ---" % (time.time() - start_time))


Processed row 0
CPU: 15.2% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5913.0MB / 24576.0MB | Load: 0.0%
Processed row 1
CPU: 17.3% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5915.0MB / 24576.0MB | Load: 0.0%
Processed row 2
CPU: 17.4% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5917.0MB / 24576.0MB | Load: 15.0%
Processed row 3
CPU: 14.7% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5917.0MB / 24576.0MB | Load: 0.0%
Processed row 4
CPU: 15.3% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5917.0MB / 24576.0MB | Load: 0.0%
Processed row 5
CPU: 13.7% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5917.0MB / 24576.0MB | Load: 0.0%
Processed row 6
CPU: 15.7% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5917.0MB / 24576.0MB | Load: 0.0%
Processed row 7
CPU: 12.1% | RAM: 37.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 5917.0MB / 24576.0MB | Load: 0.0%
Processed row 8
CPU: 13.7% | RAM: 37.8%
GPU: NVIDIA GeF

In [20]:
import re


extracted_labels = []
for item in pred:
    # Use regular expression to find "Attack" or "Normal" (case-insensitive)
    match = re.search(r'"?(Dynamic|Static)"?', item, re.IGNORECASE)
    if match:
        extracted_labels.append(match.group(1).capitalize())  # Capitalize for consistency
    else:
        extracted_labels.append('Static')

print(extracted_labels)

['Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic']


In [21]:
len(extracted_labels)

50

In [23]:
from sklearn.metrics import accuracy_score
a=accuracy_score(df2['Trigger Type'],extracted_labels)
print(a)
from sklearn.metrics import accuracy_score
a=confusion_matrix(df2['Trigger Type'],extracted_labels)
print(a)


0.54
[[26  0]
 [23  1]]


In [24]:
import time
import psutil
import GPUtil
import ollama

# Function to monitor CPU and RAM
def log_cpu_memory_usage():
    cpu = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory().percent
    print(f"CPU: {cpu}% | RAM: {memory}%")

# Function to monitor GPU (NVIDIA)
def log_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        for gpu in gpus:
            print(f"GPU: {gpu.name} | Memory Used: {gpu.memoryUsed}MB / {gpu.memoryTotal}MB | Load: {gpu.load * 100:.1f}%")
    else:
        print("No GPU detected or not supported.")

# Your original timing code
start_time = time.time()
pred = []

for i in range(len(df2)):
    response = ollama.chat(
        model="mistral",
        messages=[{"role": "user", "content": create_rag_prompt(df2.iloc[i])}]
    )
    prediction = response['message']['content']
    pred.append(prediction)
    print(f"Processed row {i}")

    # Log resource usage after each prediction
    log_cpu_memory_usage()
    log_gpu_usage()

print("--- Total time: %.2f seconds ---" % (time.time() - start_time))


Processed row 0
CPU: 22.5% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6771.0MB / 24576.0MB | Load: 0.0%
Processed row 1
CPU: 26.9% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 2
CPU: 24.1% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 3
CPU: 23.4% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 4
CPU: 19.7% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 5
CPU: 17.9% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 6
CPU: 24.0% | RAM: 43.1%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 7
CPU: 24.7% | RAM: 43.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 6775.0MB / 24576.0MB | Load: 0.0%
Processed row 8
CPU: 19.8% | RAM: 43.0%
GPU: NVIDIA GeFo

In [26]:
import re


extracted_labels = []
for item in pred:
    # Use regular expression to find "Attack" or "Normal" (case-insensitive)
    match = re.search(r'"?(Dynamic|Static)"?', item, re.IGNORECASE)
    if match:
        extracted_labels.append(match.group(1).capitalize())  # Capitalize for consistency
    else:
        extracted_labels.append('Static')

print(extracted_labels)

['Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static']


In [27]:
from sklearn.metrics import accuracy_score
a=accuracy_score(df2['Trigger Type'],extracted_labels)
print(a)
from sklearn.metrics import accuracy_score
a=confusion_matrix(df2['Trigger Type'],extracted_labels)
print(a)


0.66
[[26  0]
 [17  7]]


In [28]:
import time
import psutil
import GPUtil
import ollama

# Function to monitor CPU and RAM
def log_cpu_memory_usage():
    cpu = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory().percent
    print(f"CPU: {cpu}% | RAM: {memory}%")

# Function to monitor GPU (NVIDIA)
def log_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        for gpu in gpus:
            print(f"GPU: {gpu.name} | Memory Used: {gpu.memoryUsed}MB / {gpu.memoryTotal}MB | Load: {gpu.load * 100:.1f}%")
    else:
        print("No GPU detected or not supported.")

# Your original timing code
start_time = time.time()
pred = []

for i in range(len(df2)):
    response = ollama.chat(
        model="llama3",
        messages=[{"role": "user", "content": create_rag_prompt(df2.iloc[i])}]
    )
    prediction = response['message']['content']
    pred.append(prediction)
    print(f"Processed row {i}")

    # Log resource usage after each prediction
    log_cpu_memory_usage()
    log_gpu_usage()

print("--- Total time: %.2f seconds ---" % (time.time() - start_time))


Processed row 0
CPU: 24.5% | RAM: 45.5%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 12849.0MB / 24576.0MB | Load: 0.0%
Processed row 1
CPU: 27.8% | RAM: 44.2%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 12849.0MB / 24576.0MB | Load: 36.0%
Processed row 2
CPU: 18.8% | RAM: 44.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 12849.0MB / 24576.0MB | Load: 0.0%
Processed row 3
CPU: 14.5% | RAM: 44.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 12849.0MB / 24576.0MB | Load: 0.0%
Processed row 4
CPU: 15.8% | RAM: 44.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 12849.0MB / 24576.0MB | Load: 0.0%
Processed row 5
CPU: 15.5% | RAM: 44.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 12849.0MB / 24576.0MB | Load: 0.0%
Processed row 6
CPU: 25.0% | RAM: 44.0%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 7075.0MB / 24576.0MB | Load: 3.0%
Processed row 7
CPU: 23.5% | RAM: 43.8%
GPU: NVIDIA GeForce RTX 3090 | Memory Used: 7279.0MB / 24576.0MB | Load: 5.0%
Processed row 8
CPU: 21.3% | RAM: 43.8%
GPU: NVID

In [29]:
import re


extracted_labels = []
for item in pred:
    # Use regular expression to find "Attack" or "Normal" (case-insensitive)
    match = re.search(r'"?(Dynamic|Static)"?', item, re.IGNORECASE)
    if match:
        extracted_labels.append(match.group(1).capitalize())  # Capitalize for consistency
    else:
        extracted_labels.append('Static')

print(extracted_labels)

['Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Static', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static', 'Static', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Dynamic', 'Static']


In [30]:
from sklearn.metrics import accuracy_score
a=accuracy_score(df2['Trigger Type'],extracted_labels)
print(a)
from sklearn.metrics import accuracy_score
a=confusion_matrix(df2['Trigger Type'],extracted_labels)
print(a)


0.7
[[26  0]
 [15  9]]


In [27]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score



# Step 2: Encode all categorical columns (except target)
label_encoders = {}
for col in df_shuffled.columns:
    if df_shuffled[col].dtype == 'object' and col != 'Trigger Type':
        le = LabelEncoder()
        df_shuffled[col] = le.fit_transform(df_shuffled[col])
        label_encoders[col] = le

# Step 3: Encode the target column
target_le = LabelEncoder()
df_shuffled['Trigger Type'] = target_le.fit_transform(df_shuffled['Trigger Type'])

# Step 4: Split features and labels
X = df_shuffled.drop('Trigger Type', axis=1)
y = df_shuffled['Trigger Type']

# Step 5: Define Logistic Regression model
model = LogisticRegression(max_iter=1000)

# Step 6: Apply K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

# Step 7: Output the results
print("Accuracy scores for each fold:", scores)
print("Average Accuracy:", scores.mean())


Accuracy scores for each fold: [0.52  0.57  0.575 0.495 0.625]
Average Accuracy: 0.5569999999999999


In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score



# Step 2: Encode all categorical columns (except target)
label_encoders = {}
for col in df_shuffled.columns:
    if df_shuffled[col].dtype == 'object' and col != 'Trigger Type':
        le = LabelEncoder()
        df_shuffled[col] = le.fit_transform(df_shuffled[col])
        label_encoders[col] = le

# Step 3: Encode the target column
target_le = LabelEncoder()
df_shuffled['Trigger Type'] = target_le.fit_transform(df_shuffled['Trigger Type'])

# Step 4: Split features and labels
X = df_shuffled.drop('Trigger Type', axis=1)
y = df_shuffled['Trigger Type']

# Step 5: Define Logistic Regression model
model = RandomForestClassifier(max_depth=30,n_estimators=300)

# Step 6: Apply K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

# Step 7: Output the results
print("Accuracy scores for each fold:", scores)
print("Average Accuracy:", scores.mean())


Accuracy scores for each fold: [0.98  0.99  0.995 0.985 0.98 ]
Average Accuracy: 0.986
